In [127]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [128]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [129]:
def readData(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [130]:
def combineDF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [131]:
def selectFarmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df_farmland = df[((df.交易標的 == "土地")&(((df.都市土地使用分區 =="農")&(df.非都市土地使用編定 !="丁種建築用地"))| ((df.非都市土地使用分區 == "一般農業區")& (df.非都市土地使用編定 !="丁種建築用地"))|((df.非都市土地使用分區 == '特定農業區')& (df.非都市土地使用編定 !="丁種建築用地")))) ]
    
    return df_farmland

In [132]:
def fillNanprice(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculateprice(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [133]:
def grouptown(df,list_all_town):
    grouped=pd.to_numeric(df_farmland['單價(元/平方公尺)']).groupby(df_farmland['鄉鎮市區'])
    town_dict = {}
   
    for key,group in grouped:
        m = grouped.count()
        town_dict[key]= m[key] 
        if key not in list_all_town:
            list_all_town.append(key)
    return town_dict


In [134]:
def transcationNum(df):
    return df['鄉鎮市區'].count()

In [135]:
def calculateprice(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [136]:
def totalcalculate(key): 
    value = sum(key.values())   
    return value

In [137]:
def averagenum(num):
    nsum = 0
    for i in num:
        nsum = nsum + i
    return nsum / len(num)

In [ ]:
list_cityfilename = ['A','B','C','D','E','F','G','H','I','J','K','M','N','O','P','Q','T','U','V','W','X','Z']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','嘉義市','新竹縣','苗栗縣','南投縣','彰化縣','新竹市','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣','金門縣','澎湖縣','連江縣']
#list_cityfilename = ['H','D','N','B','T']
#list_city = ['桃園縣','台南市','彰化縣','台中市','屏東縣']
i=0
taiwan_transacation = {}#各縣市各年交易量
for c in list_cityfilename: #every city
    
    s = 4 # from 101-s4 to 107-s2
    town_tran_dist = {}
    鄉鎮逐年交易量 = {}
    list_all_town = []
    for y in range(101,108): #every year
        df = pd.DataFrame()
        while s <= 4:
            # read data
            df_s =  readData(str(y),str(s),c)
            df = combineDF(df,df_s)

            if y == 107 and s == 2:
                break
            s = s+1
        s=1
        # select farmland
        df_farmland = selectFarmland(df)
        # fill nan price
        df_farmland = fillNanprice(df_farmland)
        #鄉鎮各年交易量
        鄉鎮逐年交易量[str(y)] = grouptown(df_farmland,list_all_town)
        #縣市各年交易量
        town_tran_dist[y] = transcationNum(df_farmland)
        
    taiwan_transacation[list_city[i]]=town_tran_dist
   
    #各鄉鎮101-107總交易量
  
    鄉鎮={}
    鄉鎮歷年交易量 = {}
    各鄉鎮加總 = {}
    for t in list_all_town:
        鄉鎮 = {}
        for y in range(101,108):
            if t in 鄉鎮逐年交易量[str(y)].keys():
                鄉鎮[y] = 鄉鎮逐年交易量[str(y)][t]
        鄉鎮歷年交易量[t] = 鄉鎮
        各鄉鎮加總[t]=totalcalculate(鄉鎮歷年交易量[t])
        
    #鄉鎮總交易量排名
    sorted_transcation = sorted(各鄉鎮加總.items(),key=lambda s:s[1], reverse=True)
    
    for s in range(len(list_all_town)):
        print("|"+str(s+1)+"|",end='')
        print(sorted_transcation[s][0]+"/"+str(sorted_transcation[s][1])+"|") 
    i = i+1



In [ ]:
y_taiwan_transcation = {}
for y in range(101,108):
    年度交易量 = []
    for c in list_city:      
        年度交易量.append(taiwan_transacation[c][y])
    y_taiwan_transcation[y] = sum(年度交易量)

#平均交易量
print("平均交易量")
print(averagenum(y_taiwan_transcation.values()))

for i in range(101,108):
    print("|"+str(i)+"|",end='')
    print(str(y_taiwan_transcation[i])+"|") 


In [ ]:
#縣市各年交易量排名
print("縣市各年交易量排名")
table_dict = {}
for y in range(101,108):
    sorted_transcation = sorted(taiwan_transacation.items(),key=lambda s:s[1][y], reverse=True)
    table_dict[y] = sorted_transcation
for i in range(0,22):
    print("|"+str(i+1)+"|",end='')
    for y in range(101,108):
        print(table_dict[y][i][0]+"/"+str(table_dict[y][i][1][y])+"|",end="") 
    print("")

In [ ]:
#各縣市101-107總交易量
print("各縣市101-107總交易量")
total_transcation = {}
for c in list_city:
    total_transcation[c] = totalcalculate(taiwan_transacation[c])

table_dict = {}
sorted_transcation = sorted(total_transcation.items(),key=lambda s:s[1], reverse=True)
print("")
print(sorted_transcation)
print("")
for i in range(0,22):
    print("|"+str(i+1)+"|",end='')
    print(sorted_transcation[i][0]+"|"+str(sorted_transcation[i][1])+"|") 